In [9]:
import pandas as pd
import nltk
import re
#Aspect extraction part
!pip install rake_nltk
from rake_nltk import Rake
nltk.download('stopwords')
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords  
from pywsd.utils import lemmatize_sentence
from nltk.corpus import stopwords 
from nltk.tokenize import word_tokenize
import numpy as np

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\hp\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [10]:
pd.set_option("mode.chained_assignment", None)

In [11]:
dataset = pd.read_excel(r"E:\0001_Studies\RnD-CustomerBehaviourAnalytics\RnD Survey Data Actual.xlsx")

In [12]:
data = dataset
data.columns

Index(['Sl. No', 'Year', 'Study Name/Product Code', 'Type of study',
       'Type of product', 'Name', 'Gender', 'Region', 'Likes', 'Dislikes',
       ...
       'CR Cool feel', 'CR Deep cleaning from head to toe',
       'CR Energizing your body', 'PI', 'AS Test product',
       'AS Regular use product', 'AP Liked purchased',
       'AP Liked Not purchased', 'AP Disliked purchased',
       'AP Disliked Not Purchased'],
      dtype='object', length=141)

In [8]:
data['Study Name/Product Code'].value_counts()

Foundation Talc - ( 541)               93
 Fairness Serum - 419.                 70
Spinz winter Cream                     69
 Fairness  Cream - (734)               65
Nyle Winter Facial Cleanser            65
Fairness Cream - ( 381)                55
Spinz BB Fairness Cream                37
NYLE BOUTIQUE MOISTURIZING LOTION      35
NB - Moisturizing Lotion               35
Face Pack - 352                        33
Biker's Men Anti-Pollution Facewash    31
Nyle Boutique Brightening cream        30
Biker's Men Shower Gel                 29
Biker's Men Oil Control Facewash       29
Biker's Men's Face Cream               18
Improved Fairness Cream                15
Name: Study Name/Product Code, dtype: int64

In [13]:
str=['softness','brightening','shiny','skin','fragrance','whitening','smoothness','aroma','spreading','season','color','cream','brightness','soft','smooth','long','fairness','non oily','bright','soft','smooth','smoothy','lasting','thickness','mild','smell','dryness','pacth','brighten','brightlook','look','freshness','fresh','yes','creamy','cleansing','moisturise','pimple','face','milky','fruit','smell','glow','olive','oil','healthy','clean','foam','quantity','purchase','repurchase','water','packaging','smudge','moisturing','pdt','sandal','quantity','talc','packing','powder','costly','sweat','control','absorb','rough','hair','tangle','dirt',
'sticky','dryness','texture','dullness','oily','dust','moisturizing','frag','moisturizer','facial','soapy','hydrate','avacado','butter','spread','moisturize','natural','flavour','tightness','scrub','colour','whiteness','circle','dark','remove','result','rash','itch','tighten','granule','irritation','thin','effect','garnier','coolness','chemical','dry','thick','whity','moisture','pollution','blemish','alovera','spot','sunscreen','sweat','qty','patchy','shining','wrinkle','shine','spreadability','sweating','pack','brand','price','pdt','market','fairness','time','nutrition','clean','moisturizer','blackness','oilyness','tanning']

In [14]:
data['Likes'].replace('\d+', "", regex=True,inplace= True )
data['Likes'] = data['Likes'].str.replace("/", " and ")
data['Likes']= data['Likes'].str.replace("[^a-zA-Z#]", " ")
data['Likes'] = data['Likes'].replace(np.nan, " ", regex=True)
data['Likess'] = data['Likes']

In [15]:
places=['dl']
#xtracting loc and location 
pattern='|'.join(places)
x=data['Likes'].str.contains(pattern)
data['Sentiment']=x.replace((True,False),('negative','positive'))
data['Likes'] = data['Likes'].str.replace("dl", " ")
data['Likes']=[x for x in data['Likes'].str.lower().str.split()]
data['Liked_Aspect']=data['Likes'].apply(lambda x: [x for x in x if x in str])

In [16]:
#extracting aspect with higher ranking phrases (top5 min)
aspect_adj_string =[]
for i in data['Likes'].values:
    try:
        r= Rake()
        r = Rake(min_length=1, max_length=2)
        a=r.extract_keywords_from_text(i)
        b=r.get_ranked_phrases()[:5]  
        aspect_adj_string.append(b)
    except:
        aspect_adj_string.append(0)
data['aspect_adj_string']=aspect_adj_string

In [17]:
data['AP Liked purchased'] = data['AP Liked purchased'].replace("/", " and ")
data['AP Liked purchased'] = data['AP Liked purchased'].str.replace("[^a-zA-Z#]", " ")
data['AP Liked purchased'] = data['AP Liked purchased'].replace(np.nan, '', regex=True)
data['AP Liked purchased']=data['AP Liked purchased'].str.lower().str.split()
data['AP_Likedpurchased_Aspect']=data['AP Liked purchased'].apply(lambda x: [x for x in x if x in str])
data.drop(columns=['Likess'],inplace=True)

In [18]:
data['AP Liked Not purchased'] = data['AP Liked Not purchased'].replace("/", " and ")
data['AP Liked Not purchased']= data['AP Liked Not purchased'].str.replace("[^a-zA-Z#]", " ")
data['AP Liked Not purchased'] = data['AP Liked Not purchased'].replace(np.nan, '', regex=True)
lemmatizer = WordNetLemmatizer()
data['AP Liked Not purchased'] = data['AP Liked Not purchased'].apply(lemmatize_sentence)
data['AP_LikedNotpurchased_Aspect']=data['AP Liked Not purchased'].apply(lambda x: [x for x in x if x in str])

In [19]:
data['AP Disliked purchased'] = data['AP Disliked purchased'].replace("/", " and ")
data['AP Disliked purchased']= data['AP Disliked purchased'].str.replace("[^a-zA-Z#]", " ")
data['AP Disliked purchased'] = data['AP Disliked purchased'].replace(np.nan, '', regex=True)
lemmatizer = WordNetLemmatizer()
data['AP Disliked purchased'] = data['AP Disliked purchased'].apply(lemmatize_sentence)
data['AP_Dislikedpurchased_Aspect']=data['AP Disliked purchased'].apply(lambda x: [x for x in x if x in str])

In [20]:
data['AP Disliked Not Purchased'] = data['AP Disliked Not Purchased'].replace("/", " and ")
data['AP Disliked Not Purchased']= data['AP Disliked Not Purchased'].str.replace("[^a-zA-Z#]", " ")
data['AP Disliked Not Purchased'] = data['AP Disliked Not Purchased'].replace(np.nan, '', regex=True)
lemmatizer = WordNetLemmatizer()
data['AP Disliked Not Purchased'] = data['AP Disliked Not Purchased'].apply(lemmatize_sentence)
data['AP_Dislikednotpurchased_Aspect']=data['AP Disliked Not Purchased'].apply(lambda x: [x for x in x if x in str])

In [21]:
data['Dislikes'] = data['Dislikes'].replace("/", " and ")
data['Dislikes']= data['Dislikes'].str.replace("[^a-zA-Z#]", " ")
data['Dislikes'] = data['Dislikes'].replace(np.nan, '', regex=True)
lemmatizer = WordNetLemmatizer()
data['Dislikes'] = data['Dislikes'].apply(lemmatize_sentence)
data['Disliked_Aspect']=data['Dislikes'].apply(lambda x: [x for x in x if x in str])

In [22]:
data['AS Test product'] = data['AS Test product'].replace("/", " and ")
data['AS Test product']= data['AS Test product'].str.replace("[^a-zA-Z#]", " ")
data['AS Test product'] = data['AS Test product'].replace(np.nan, '', regex=True)
lemmatizer = WordNetLemmatizer()
data['AS Test product'] = data['AS Test product'].apply(lemmatize_sentence)
data['AS_Testproduct_Aspect']=data['AS Test product'].apply(lambda x: [x for x in x if x in str])

## Aspect Columns are

#### 1. 'AP_Likedpurchased_Aspect'
#### 2. 'AP_Likednotpurchased_Aspect'
#### 3. 'AP_Dislikedpurchased_Aspect'
#### 4. 'AP_Dislikednotpurchased_Aspect'
#### 5. 'Disliked_Aspect' and 'Liked_Aspect'
#### 6. 'AS_Testproduct_Aspect'

In [176]:
Product = data["Study Name/Product Code"].unique()
Product

array(['Improved Fairness Cream ', ' Fairness Serum - 419.',
       ' Fairness  Cream - (734)', 'Fairness Cream - ( 381)',
       'Face Pack - 352', 'Foundation Talc - ( 541)',
       'Spinz BB Fairness Cream', 'NB - Moisturizing Lotion',
       'Nyle Winter Facial Cleanser',
       "Biker's Men Anti-Pollution Facewash",
       "Biker's Men Oil Control Facewash", "Biker's Men Shower Gel",
       "Biker's Men's Face Cream", 'NYLE BOUTIQUE MOISTURIZING LOTION',
       'Spinz winter Cream ', 'Nyle Boutique Brightening cream'],
      dtype=object)

In [177]:
from collections import Counter
from functools import reduce

def get_frequncy(df_1,Product_list):
    final_df = pd.DataFrame()    
    
    for i in Product_list:
        
        df = df_1[df_1['Study Name/Product Code']==i]
        #print('Product is',i)
        like_aspect_list = df['Liked_Aspect']
        dislike_aspect_list = df['Disliked_Aspect']
        Likedpurchased_aspect_list = df['AP_Likedpurchased_Aspect']
        LikedNotpurchased_aspect_list = df['AP_LikedNotpurchased_Aspect']
        Dislikedpurchased_aspect_list = df['AP_Dislikedpurchased_Aspect']
        Dislikednotpurchased_aspect_list = df['AP_Dislikednotpurchased_Aspect']
        Testproduct_aspect_list = df['AS_Testproduct_Aspect']

        like_aspect_list = [item for items in like_aspect_list for item in items]
        dislike_aspect_list = [item for items in dislike_aspect_list for item in items]
        Likedpurchased_aspect_list = [item for items in Likedpurchased_aspect_list for item in items]
        Likednotpurchased_aspect_list = [item for items in LikedNotpurchased_aspect_list for item in items]
        Dislikedpurchased_aspect_list = [item for items in Dislikedpurchased_aspect_list for item in items]
        Dislikednotpurchased_aspect_list = [item for items in Dislikednotpurchased_aspect_list for item in items]
        Testproduct_aspect_list = [item for items in Testproduct_aspect_list for item in items]

        like_aspect_count = Counter(like_aspect_list).items()
        dislike_aspect_count = Counter(dislike_aspect_list).items()
        Likedpurchased_aspect_count = Counter(Likedpurchased_aspect_list).items()
        Likednotpurchased_aspect_count = Counter(Likednotpurchased_aspect_list).items()
        Dislikedpurchased_aspect_count = Counter(Dislikedpurchased_aspect_list).items()
        Dislikednotpurchased_aspect_count = Counter(Dislikednotpurchased_aspect_list).items()
        Testproduct_aspect_count = Counter(Testproduct_aspect_list).items()

        #Check for empty rows in dfframe
        def fill_empty_dfframe(aspect):
            if(all(x == [] for x in aspect)==True):
                value = [{'Empty','Value'}]
            elif(all(x == [] for x in aspect)==False):
                value = aspect
            return value

        like_aspect_count = fill_empty_dfframe(like_aspect_count)
        dislike_aspect_count = fill_empty_dfframe(dislike_aspect_count)
        Likedpurchased_aspect_count = fill_empty_dfframe(Likedpurchased_aspect_count)
        Likednotpurchased_aspect_count = fill_empty_dfframe(Likednotpurchased_aspect_count)
        Dislikedpurchased_aspect_count = fill_empty_dfframe(Dislikedpurchased_aspect_count)
        Dislikednotpurchased_aspect_count = fill_empty_dfframe(Dislikednotpurchased_aspect_count)
        Testproduct_aspect_count = fill_empty_dfframe(Testproduct_aspect_count)

        Like_aspect_Frequency = pd.DataFrame.from_dict(like_aspect_count)
        Like_aspect_Frequency.rename(columns={0:'Word',1:'Like_aspect_Frequency'},inplace=True)
        dislike_aspect_Frequency = pd.DataFrame.from_dict(dislike_aspect_count)
        dislike_aspect_Frequency.rename(columns={0:'Word',1:'Dislike_aspect_Frequency'},inplace=True)
        Likedpurchased_Frequency = pd.DataFrame.from_dict(Likedpurchased_aspect_count)
        Likedpurchased_Frequency.rename(columns={0:'Word',1:'Likedpurchased_Frequency'},inplace=True)
        Likednotpurchased_Frequency = pd.DataFrame.from_dict(Likednotpurchased_aspect_count)
        Likednotpurchased_Frequency.rename(columns={0:'Word',1:'Likednotpurchased_Frequency'},inplace=True)
        Dislikedpurchased_Frequency = pd.DataFrame.from_dict(Dislikedpurchased_aspect_count)
        Dislikedpurchased_Frequency.rename(columns={0:'Word',1:'Dislikedpurchased_Frequency'},inplace=True)
        Dislikednotpurchased_Frequency = pd.DataFrame.from_dict(Dislikednotpurchased_aspect_count)
        Dislikednotpurchased_Frequency.rename(columns={0:'Word',1:'Dislikednotpurchased_Frequency'},inplace=True)
        Testproduct_Frequency = pd.DataFrame.from_dict(Testproduct_aspect_count)
        Testproduct_Frequency.rename(columns={0:'Word',1:'Testproduct_Frequency'},inplace=True)

        df_frames = [Like_aspect_Frequency,dislike_aspect_Frequency,Likedpurchased_Frequency,Dislikedpurchased_Frequency,
                    Dislikednotpurchased_Frequency,Testproduct_Frequency,Likednotpurchased_Frequency]
        Aspect_df = reduce(lambda  left,right: pd.merge(left,right,on=['Word'],how='outer'), df_frames)

        Aspect_df["Study Name/Product Code"] = np.repeat(i,len(Aspect_df))
        final_df = final_df.append(Aspect_df)
    final_df.fillna(0,inplace=True)
    return final_df

In [179]:
final_output = get_frequncy(data,Product)
#final_output.sample(10)
    final_output.to_excel(r"E:\0001_Studies\RnD-CustomerBehaviourAnalytics\output\final_NLP_output.xlsx",index=False)